In [ ]:
from dhanhq import dhanhq
import yaml
import pandas  as pd
import numpy as np



## Authentication

In [ ]:
CONFIG_FILE_PATH = r"..\\configs\\config.yaml" #This has the API key and client id
with open(CONFIG_FILE_PATH,'r') as config:
    api_config = yaml.safe_load(config)
APPLICAITON_ID = api_config.get('api_config')[0]
CLIENT_ID = api_config.get('api_config')[1]
ACCESS_TOKEN = api_config.get('api_config')[2]
EXP_HOURS = api_config.get('api_config')[3]
dhan = dhanhq(CLIENT_ID,ACCESS_TOKEN)

## Getting other variables from configuration

In [ ]:
OTHER_CONFIG_FILE_PATH = r"..\\configs\\other_config.yaml" # This we will configure all other variables
with open(OTHER_CONFIG_FILE_PATH,'r') as other_configs:
    gen_config = yaml.safe_load(other_configs)

In [ ]:
strike_size = gen_config.get('strike_size')
underlying = gen_config.get('index_file_metadata')
options = gen_config.get('options_file_metadata')

In [129]:
underlying = pd.read_excel('underlying.xlsx')
options = pd.read_excel('Options.xlsx')


In [130]:
UNDERLYING_INDEX = "NIFTY"

In [183]:
def get_otms_itma(SPOT):
    INDEX_CE_ITM = {"ITM" :
    {
        x : SPOT - ((SPOT%strike_size[UNDERLYING_INDEX]) + (x*strike_size[UNDERLYING_INDEX])) for x in range(10)
    }
    }
    INDEX_PE_ITM =  {"ITM" :
    {
        x : SPOT - ((SPOT%strike_size[UNDERLYING_INDEX]) - ((x+1)*strike_size[UNDERLYING_INDEX])) for x in range(10)
    }
    }
    INDEX_ATM = SPOT - (SPOT%strike_size[UNDERLYING_INDEX])
    return({"CE" : INDEX_CE_ITM, "PE" : INDEX_PE_ITM, "ATM":INDEX_ATM})
    

In [184]:
def get_index_ltp(dhanO, INDEX):
    INDEX_ID = str(underlying.loc[(underlying.SEM_TRADING_SYMBOL == INDEX),['SEM_SMST_SECURITY_ID']].squeeze())
    INDEX_LTP = dhanO.ohlc_data(securities = {"IDX_I":[int(INDEX_ID)]})['data']['data']['IDX_I'][f'{INDEX_ID}']['last_price']
    STRIKE_PRICES = get_otms_itma(INDEX_LTP)
    return {"INDEX": INDEX, "SPOT": INDEX_LTP, "STRIKES":STRIKE_PRICES}

    

In [185]:
INDEX_LTP_STRIKE = get_index_ltp(dhan,UNDERLYING_INDEX)

In [186]:
INDEX_LTP_STRIKE

{'INDEX': 'NIFTY',
 'SPOT': 23883.45,
 'STRIKES': {'CE': {'ITM': {0: 23850.0,
    1: 23800.0,
    2: 23750.0,
    3: 23700.0,
    4: 23650.0,
    5: 23600.0,
    6: 23550.0,
    7: 23500.0,
    8: 23450.0,
    9: 23400.0}},
  'PE': {'ITM': {0: 23900.0,
    1: 23950.0,
    2: 24000.0,
    3: 24050.0,
    4: 24100.0,
    5: 24150.0,
    6: 24200.0,
    7: 24250.0,
    8: 24300.0,
    9: 24350.0}},
  'ATM': 23850.0}}

In [187]:
23883.45%50

33.45000000000073

In [188]:
23883 - 33

23850